In [1]:
from pyspark.sql import Row
from pyspark.sql.functions import col 
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import time

In [2]:
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

train_positive_pairs = sc.textFile(base_path + "new_positive_pairs_features/day*/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
train_unlabled_pairs = sc.textFile(base_path + "new_unlabled_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_positive_pairs = sc.textFile(base_path + "test_positive_pairs/part-*")

test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_pairs_features_50w = sc.textFile(base_path + "test_pairs_features_50w/part-*").map(lambda x: eval(x))

In [3]:
"""test_positive_pairs_list = test_positive_pairs.collect()   # 第四天数据中的正例对
test_data_4thday = test_pairs_features_50w.map(lambda x: Row(label=1.0 if x[0] in test_positive_pairs_list else 0.0,\
                                                         features=Vectors.dense(x[1]))).toDF()"""

In [3]:
"""test_positive_pairs_list = test_positive_pairs.collect()   # 第四天数据中的正例对
#test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
test_pos_features = test_pairs_features.filter(lambda x: x[0] in test_positive_pairs_list)
test_neg_features = test_pairs_features.filter(lambda x: x[0] not in test_positive_pairs_list).takeSample(False, 80000)"""

'test_positive_pairs_list = test_positive_pairs.collect()   # 第四天数据中的正例对\n#test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))\ntest_pos_features = test_pairs_features.filter(lambda x: x[0] in test_positive_pairs_list)\ntest_neg_features = test_pairs_features.filter(lambda x: x[0] not in test_positive_pairs_list).takeSample(False, 80000)'

In [18]:
"""test_pos_features.saveAsTextFile(base_path + "test_pos_features")
sc.parallelize(test_neg_features).saveAsTextFile(base_path + "test_neg_features_8w")"""

In [14]:
"""test_data_4thday = test_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(sc.parallelize(test_neg_features).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()"""

In [5]:
test_pos_features = sc.textFile(base_path + "test_pos_features/part-*").map(lambda x: eval(x))
test_neg_features = sc.textFile(base_path + "test_neg_features_8w/part-*").map(lambda x: eval(x))
test_data_4thday = test_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(test_neg_features.map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

In [15]:
test_data_4thday.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
+--------------------+-----+
only showing top 3 rows



In [12]:
len(test_data_4thday.first().features)

194

In [6]:
y_label = [i.label for i in test_data_4thday.select('label').collect()]

In [7]:
test_data_4thday_length = len(y_label) #test_data_4thday.count()
print(test_data_4thday_length)

157653


In [8]:
num1 = num0 = 0
for i in y_label:
    if i == 1.0:
        num1 += 1
    if i == 0.0:
        num0 += 1
print(num0, num1, num0/num1)

80000 77653 1.0302242025420782


In [3]:

for alfa in [0.35]: #0.30
    with open('qq_PU_alfa_1_v331.log', 'a+') as log_file1:
    # 
        log_file1.write(time.asctime() + '\n')
        log_file1.write("\n===GET RN===\n" + '\n')
        log_file1.write( "==========alfa: " + str(alfa) + "============\n")
        print(time.asctime())
        # train_unlabeled -> train_neg
        #train_negative_pairs = train_unlabled_pairs.takeSample(False, int(alfa * 8268837))
        #sc.parallelize(train_negative_pairs).coalesce(20).saveAsTextFile(base_path + "wbb_train_negative_pairs-" + str(alfa))
        train_negative_pairs = sc.textFile(base_path + "wbb_train_negative_pairs-" + str(alfa)).map(lambda x: eval(x))#.collect()

        #log_file1.write("length of train_negative_pairs: " + str(len(train_negative_pairs)) + '\n')
        #print(time.asctime())
        log_file1.write(time.asctime() + '\n')

        # ===============================================
        # with PU
        # train_positive_pairs.count() = 235586
        blta = 0.8  # 从train pos 里面取一部分加到 neg 里面组成PU数据
        pnsplit = int(235586*blta)   # pnsplit以前的部分作为pu_pos, 以后的部分作为pu_neg
        pu_pos_features = train_positive_pairs.collect()[:pnsplit]
        pu_neg_features = train_negative_pairs.collect() + train_positive_pairs.collect()[pnsplit:]
        #pu_spy = [Vectors.dense(x[1]) for x in train_positive_pairs.collect()[pnsplit:]]
        
        log_file1.write("length of pu_pos_features: " + str(len(pu_pos_features)) + '\n')
        log_file1.write("length of pu_neg_features: " + str(len(pu_neg_features)) + '\n')
        log_file1.write(time.asctime() + '\n\n')
        print(time.asctime())

        gamma=0.3  # 从PU数据里面取一部分训练，另一部分作为测试
        pu_train_pos = sc.parallelize(pu_pos_features).takeSample(False, int(len(pu_pos_features) * gamma))  # list
        pu_test_pos = sc.parallelize(pu_pos_features).subtractByKey(sc.parallelize(pu_train_pos)).collect()
        log_file1.write("length of pu_train_pos: " + str(len(pu_train_pos)) + '\n')
        log_file1.write("length of pu_test_pos: " + str(len(pu_test_pos)) + '\n')
        pu_train_neg = sc.parallelize(pu_neg_features, 50).takeSample(False, int(len(pu_neg_features) * gamma))  # list
        pu_test_neg = sc.parallelize(pu_neg_features).subtractByKey(sc.parallelize(pu_train_neg)).collect()
        log_file1.write("length of pu_train_neg: " + str(len(pu_train_neg)) + '\n')
        log_file1.write("length of pu_test_neg: " + str(len(pu_test_neg)) + '\n')
        log_file1.write(time.asctime() + '\n\n')
        print(time.asctime())

        pu_train_data = sc.parallelize(pu_train_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(sc.parallelize(pu_train_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

        pu_test_data = sc.parallelize(pu_test_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1]), pair=x[0]))\
        .union(sc.parallelize(pu_test_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]), pair=x[0]))).toDF()

        
        ### LR
        log_file1.write("###### LogisticRegression ######\n")
        publr = LogisticRegression().fit(pu_train_data)
        pu_test_data_preRes_LR = publr.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

        predict_to_0_LR = pu_test_data_preRes_LR.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_LR = predict_to_0_LR.map(lambda x: x.probability[0]).collect()
        
        prob_LR_max = max(predict_to_0_prob_LR)
        prob_LR_mean = sum(predict_to_0_prob_LR)/len(predict_to_0_prob_LR)
        print(len(predict_to_0_prob_LR), prob_LR_max, min(predict_to_0_prob_LR), prob_LR_mean)
        log_file1.write("len(predict_to_0_prob_LR) ++ max(predict_to_0_prob_LR) ++ min(predict_to_0_prob_LR) ++ mean(predict_to_0_prob_LR) \n")
        log_file1.write(str(len(predict_to_0_prob_LR)) + ', ' + str(prob_LR_max) + ', ' + str(min(predict_to_0_prob_LR)) + ', ' + str(prob_LR_mean) + '\n')
        log_file1.write(time.asctime() + '\n')
        
        #RN_LR_max = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] >= prob_LR_max).map(lambda x: x.features).collect()
        RN_LR_mean = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] > prob_LR_mean).map(lambda x: x.features).takeSample(False, 240000)
        #log_file1.write('len(RN_LR_max): '+str(len(RN_LR_max))+'\n')
        #log_file1.write("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
        #print('len(RN_LR_max): '+str(len(RN_LR_max))+'\n')
        #print("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
        #sc.parallelize(RN_LR_max).saveAsTextFile(base_path + "RN/RN_LR_max-" + str(alfa))
        sc.parallelize(RN_LR_mean).saveAsTextFile(base_path + "RN/RN_LR_mean-" + str(alfa))
        
        """    
        ### NB
        log_file1.write("###### NaiveBayes ######\n")
        pubNB = NaiveBayes().fit(pu_train_data)
        pu_test_data_preRes_NB = pubNB.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

        predict_to_0_NB = pu_test_data_preRes_NB.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_NB = predict_to_0_NB.map(lambda x: x.probability[0]).collect()
        
        prob_NB_max = max(predict_to_0_prob_NB)
        prob_NB_mean = sum(predict_to_0_prob_NB)/len(predict_to_0_prob_NB)
        print(len(predict_to_0_prob_NB), prob_NB_max, min(predict_to_0_prob_NB), prob_NB_mean)
        log_file1.write("len(predict_to_0_prob_NB) ++ max(predict_to_0_prob_NB) ++ min(predict_to_0_prob_NB) ++ mean(predict_to_0_prob_NB) \n")
        log_file1.write(str(len(predict_to_0_prob_NB)) + ', ' + str(prob_NB_max) + ', ' + str(min(predict_to_0_prob_NB)) + ', ' + str(prob_NB_mean) + '\n')
        log_file1.write(time.asctime() + '\n')
        
        RN_NB_max = pu_test_data_preRes_NB.rdd.filter(lambda x: x.probability[0] >= prob_NB_max).map(lambda x: x.features).collect()
        RN_NB_mean = pu_test_data_preRes_NB.rdd.filter(lambda x: x.probability[0] > prob_NB_mean).map(lambda x: x.features).collect()
        log_file1.write('len(RN_NB_max): '+str(len(RN_NB_max))+'\n')
        log_file1.write("len(RN_NB_mean):" + str(len(RN_NB_mean))+'\n')
        print('len(RN_NB_max): '+str(len(RN_NB_max))+'\n')
        print("len(RN_NB_mean):" + str(len(RN_NB_mean))+'\n')
        sc.parallelize(RN_NB_max).saveAsTextFile(base_path + "RN/RN_NB_max-" + str(alfa))
        sc.parallelize(RN_NB_mean).saveAsTextFile(base_path + "RN/RN_NB_mean-" + str(alfa))
              
        ### DT
        log_file1.write("###### DecisionTreeClassifier ######\n")
        pubDT = DecisionTreeClassifier().fit(pu_train_data)
        pu_test_data_preRes_DT = pubDT.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer") 

        predict_to_0_DT = pu_test_data_preRes_DT.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_DT = predict_to_0_DT.map(lambda x: x.probability[0]).collect()
        
        prob_DT_max = max(predict_to_0_prob_DT)
        prob_DT_mean = sum(predict_to_0_prob_DT)/len(predict_to_0_prob_DT)
        print(len(predict_to_0_prob_DT), prob_DT_max, min(predict_to_0_prob_DT), prob_DT_mean)
        log_file1.write("len(predict_to_0_prob_DT) ++ max(predict_to_0_prob_DT) ++ min(predict_to_0_prob_DT) ++ mean(predict_to_0_prob_DT) \n")
        log_file1.write(str(len(predict_to_0_prob_DT)) + ', ' + str(prob_DT_max) + ', ' + str(min(predict_to_0_prob_DT)) + ', ' + str(prob_DT_mean) + '\n')
        log_file1.write(time.asctime() + '\n')
        
        #RN_DT_max = pu_test_data_preRes_DT.rdd.filter(lambda x: x.probability[0] >= prob_DT_max).map(lambda x: x.features).collect()
        RN_DT_mean = pu_test_data_preRes_DT.rdd.filter(lambda x: x.probability[0] > prob_DT_mean).map(lambda x: x.features).takeSample(False, 240000)
        #log_file1.write('len(RN_DT_max): '+str(len(RN_DT_max))+'\n')
        #log_file1.write("len(RN_DT_mean):" + str(len(RN_DT_mean))+'\n')
        #print('len(RN_DT_max): '+str(len(RN_DT_max))+'\n')
        #print("len(RN_DT_mean):" + str(len(RN_DT_mean))+'\n')
        #sc.parallelize(RN_DT_max).saveAsTextFile(base_path + "RN/RN_DT_max-" + str(alfa))
        sc.parallelize(RN_DT_mean).saveAsTextFile(base_path + "RN/RN_DT_mean-" + str(alfa))
            
        ### RandomForestClassifier
        log_file1.write("###### RandomForestClassifier ######\n")
        pubRF = RandomForestClassifier().fit(pu_train_data)
        pu_test_data_preRes_RF = pubRF.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

        predict_to_0_RF = pu_test_data_preRes_RF.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_RF = predict_to_0_RF.map(lambda x: x.probability[0]).collect()
        
        prob_RF_max = max(predict_to_0_prob_RF)
        prob_RF_mean = sum(predict_to_0_prob_RF)/len(predict_to_0_prob_RF)
        print(len(predict_to_0_prob_RF), prob_RF_max, min(predict_to_0_prob_RF), prob_RF_mean)
        log_file1.write("len(predict_to_0_prob_RF) ++ max(predict_to_0_prob_RF) ++ min(predict_to_0_prob_RF) ++ mean(predict_to_0_prob_RF) \n")
        log_file1.write(str(len(predict_to_0_prob_RF)) + ', ' + str(prob_RF_max) + ', ' + str(min(predict_to_0_prob_RF)) + ', ' + str(prob_RF_mean) + '\n')
        log_file1.write(time.asctime() + '\n')
        
        #RN_RF_max = pu_test_data_preRes_RF.rdd.filter(lambda x: x.probability[0] >= prob_RF_max).map(lambda x: x.features).collect()
        RN_RF_mean = pu_test_data_preRes_RF.rdd.filter(lambda x: x.probability[0] > prob_RF_mean).map(lambda x: x.features).takeSample(False, 240000)
        #log_file1.write('len(RN_RF_max): '+str(len(RN_RF_max))+'\n')
        #log_file1.write("len(RN_RF_mean):" + str(len(RN_RF_mean))+'\n')
        #print('len(RN_RF_max): '+str(len(RN_RF_max))+'\n')
        #print("len(RN_RF_mean):" + str(len(RN_RF_mean))+'\n')
        #sc.parallelize(RN_RF_max).saveAsTextFile(base_path + "RN/RN_RF_max-" + str(alfa))
        sc.parallelize(RN_RF_mean).saveAsTextFile(base_path + "RN/RN_RF_mean-" + str(alfa))
        """

Tue Feb 28 12:49:06 2017
Tue Feb 28 12:52:12 2017
Tue Feb 28 13:05:21 2017
95740 0.999999999936 0.500002544444 0.824591069664


157653


In [9]:
with open('qq_PU_alfa_2_v3321.log', 'a+') as log_file2:
    for alfa in [0.35]: # 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, , 0.40]: #, 
        log_file2.write('\n' + time.asctime() + '\n')
        log_file2.write("===Predict with RN(Sample)===\n" + '\n')
        log_file2.write( "==========alfa: " + str(alfa) + "============\n")
        print(time.asctime())
        
        # LR
        log_file2.write('###### LogisticRegression #######\n')
        RN_LR = sc.parallelize(sc.textFile(base_path + "RN/RN_LR_mean-" + str(alfa)).map(lambda x: eval(x)).takeSample(False, 240000))
        print(RN_LR.count())
        PU_train_data_LR = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_LR.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        lr = LogisticRegression().fit(PU_train_data_LR)
        predict_4th_day_LR = lr.transform(test_data_4thday.select('features'))
        
        y_predict_PU_LR = [i.prediction for i in predict_4th_day_LR.select('prediction').collect()]
        print(len(y_predict_PU_LR))
        recall_score_PU_LR = recall_score(y_label, y_predict_PU_LR)
        precision_score_PU_LR = precision_score(y_label, y_predict_PU_LR)
        f1_score_PU_LR = f1_score(y_label, y_predict_PU_LR)
        tp_LR = 0
        tn_LR = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_LR[i] == 1.0:
                tp_LR += 1
            if y_label[i] == 0.0 and y_predict_PU_LR[i] == 0.0:
                tn_LR += 1
        accuracy_LR = (tp_LR + tn_LR)/test_data_4thday_length
        
        print(recall_score_PU_LR, precision_score_PU_LR, f1_score_PU_LR, accuracy_LR)

        log_file2.write('\n')
        log_file2.write("recall_score_PU_LR: " + str(recall_score_PU_LR) + '\n')
        log_file2.write("precision_score_PU_LR: " + str(precision_score_PU_LR) + '\n')
        log_file2.write("f1_score_PU_LR: " + str(f1_score_PU_LR) + '\n')
        log_file2.write("accuracy_LR: " + str(accuracy_LR) + '\n')
        log_file2.write(time.asctime() + '\n')
        log_file2.write('\n')

        print(time.asctime())
        
        """# NB
        log_file2.write('###### NaiveBayes #######\n')
        RN_NB = sc.parallelize(sc.textFile(base_path + "RN/RN_NB_mean-" + str(alfa)).map(lambda x: eval(x)).takeSample(False, 240000))
        
        PU_train_data_NB = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_NB.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        NB = NaiveBayes().fit(PU_train_data_NB)
        predict_4th_day_NB = NB.transform(test_data_4thday.select('features'))
        
        y_predict_PU_NB = [i.prediction for i in predict_4th_day_NB.select('prediction').collect()]
        
        recall_score_PU_NB = recall_score(y_label, y_predict_PU_NB)
        precision_score_PU_NB = precision_score(y_label, y_predict_PU_NB)
        f1_score_PU_NB = f1_score(y_label, y_predict_PU_NB)
        tp_NB = 0
        tn_NB = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_NB[i] == 1.0:
                tp_NB += 1
            if y_label[i] == 0.0 and y_predict_PU_NB[i] == 0.0:
                tn_NB += 1
        accuracy_NB = (tp_NB + tn_NB)/test_data_4thday_length
        
        print(recall_score_PU_NB, precision_score_PU_NB, f1_score_PU_NB, accuracy_NB)

        log_file2.write('\n')
        log_file2.write("recall_score_PU_NB: " + str(recall_score_PU_NB) + '\n')
        log_file2.write("precision_score_PU_NB: " + str(precision_score_PU_NB) + '\n')
        log_file2.write("f1_score_PU_NB: " + str(f1_score_PU_NB) + '\n')
        log_file2.write("accuracy_NB: " + str(accuracy_NB) + '\n')
        log_file2.write(time.asctime() + '\n')
        log_file2.write('\n')

        print(time.asctime())
        
        # DT
        log_file2.write('###### DecisionTreeClassifier #######\n')
        RN_DT = sc.parallelize(sc.textFile(base_path + "RN/RN_DT_mean-" + str(alfa)).map(lambda x: eval(x)).takeSample(False, 240000))
        
        PU_train_data_DT = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_DT.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        DT = DecisionTreeClassifier().fit(PU_train_data_DT)
        predict_4th_day_DT = DT.transform(test_data_4thday.select('features'))
        
        y_predict_PU_DT = [i.prediction for i in predict_4th_day_DT.select('prediction').collect()]
        
        recall_score_PU_DT = recall_score(y_label, y_predict_PU_DT)
        precision_score_PU_DT = precision_score(y_label, y_predict_PU_DT)
        f1_score_PU_DT = f1_score(y_label, y_predict_PU_DT)
        tp_DT = 0
        tn_DT = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_DT[i] == 1.0:
                tp_DT += 1
            if y_label[i] == 0.0 and y_predict_PU_DT[i] == 0.0:
                tn_DT += 1
        accuracy_DT = (tp_DT + tn_DT)/test_data_4thday_length
        
        print(recall_score_PU_DT, precision_score_PU_DT, f1_score_PU_DT, accuracy_DT)

        log_file2.write('\n')
        log_file2.write("recall_score_PU_DT: " + str(recall_score_PU_DT) + '\n')
        log_file2.write("precision_score_PU_DT: " + str(precision_score_PU_DT) + '\n')
        log_file2.write("f1_score_PU_DT: " + str(f1_score_PU_DT) + '\n')
        log_file2.write("accuracy_DT: " + str(accuracy_DT) + '\n')
        log_file2.write(time.asctime() + '\n')
        log_file2.write('\n')

        print(time.asctime())
        
        # RF
        log_file2.write('###### RandomForestClassifier #######\n')
        RN_RF = sc.parallelize(sc.textFile(base_path + "RN/RN_RF_mean-" + str(alfa)).map(lambda x: eval(x)).takeSample(False, 240000))
        
        PU_train_data_RF = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_RF.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        RF = RandomForestClassifier().fit(PU_train_data_RF)
        predict_4th_day_RF = RF.transform(test_data_4thday.select('features'))
        
        y_predict_PU_RF = [i.prediction for i in predict_4th_day_RF.select('prediction').collect()]
        
        recall_score_PU_RF = recall_score(y_label, y_predict_PU_RF)
        precision_score_PU_RF = precision_score(y_label, y_predict_PU_RF)
        f1_score_PU_RF = f1_score(y_label, y_predict_PU_RF)
        tp_RF = 0
        tn_RF = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_RF[i] == 1.0:
                tp_RF += 1
            if y_label[i] == 0.0 and y_predict_PU_RF[i] == 0.0:
                tn_RF += 1
        accuracy_RF = (tp_RF + tn_RF)/test_data_4thday_length
        
        print(recall_score_PU_RF, precision_score_PU_RF, f1_score_PU_RF, accuracy_RF)

        log_file2.write('\n')
        log_file2.write("recall_score_PU_RF: " + str(recall_score_PU_RF) + '\n')
        log_file2.write("precision_score_PU_RF: " + str(precision_score_PU_RF) + '\n')
        log_file2.write("f1_score_PU_RF: " + str(f1_score_PU_RF) + '\n')
        log_file2.write("accuracy_RF: " + str(accuracy_RF) + '\n')
        log_file2.write(time.asctime() + '\n')
        log_file2.write('\n')

        print(time.asctime())"""

Tue Feb 28 15:57:38 2017
240000
157653
0.926223069295 0.771435313298 0.841772633378 0.8284904188312306
Tue Feb 28 16:04:37 2017


In [3]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.get("spark.driver.maxResultSize")
"""# In Jupyter you have to stop the current context first
sc.stop()

# Create new config
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "8g"))

# Create new context
sc = SparkContext(conf=conf)"""

'# In Jupyter you have to stop the current context first\nsc.stop()\n\n# Create new config\nconf = (SparkConf()\n    .set("spark.driver.maxResultSize", "8g"))\n\n# Create new context\nsc = SparkContext(conf=conf)'

In [4]:
conf.get("spark.executor.memory")

'18g'

In [5]:
conf.get("spark.executor.cores")

In [7]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [13]:
log_file1 = open('qq_PU_alfa_1_v3.log', 'a+')

In [15]:
log_file1.write("========================\n")

25

In [9]:
alfa = 0.05
train_negative_pairs = sc.textFile(base_path + "wbb_train_negative_pairs-" + str(alfa)).map(lambda x: eval(x)).collect()

log_file1.write("length of train_negative_pairs: " + str(len(train_negative_pairs)) + '\n')
print(time.asctime())
log_file1.write(time.asctime() + '\n')

# ===============================================
# with PU
blta = 0.8  # 从train pos 里面取一部分加到 neg 里面组成PU数据
pnsplit = int(train_positive_pairs.count()*blta)   # pnsplit以前的部分作为pu_pos, 以后的部分作为pu_neg
pu_pos_features = train_positive_pairs.collect()[:pnsplit]
pu_neg_features = train_negative_pairs + train_positive_pairs.collect()[pnsplit:]

log_file1.write("length of pu_pos_features: " + str(len(pu_pos_features)) + '\n')
log_file1.write("length of pu_neg_features: " + str(len(pu_neg_features)) + '\n')
log_file1.write(time.asctime() + '\n\n')
print(time.asctime())

gamma=0.3  # 从PU数据里面取一部分训练，另一部分作为测试
pu_train_pos = sc.parallelize(pu_pos_features).takeSample(False, int(len(pu_pos_features) * gamma))  # list
pu_test_pos = sc.parallelize(pu_pos_features).subtractByKey(sc.parallelize(pu_train_pos)).collect()
#log_file1.write("length of pu_train_pos: " + str(len(pu_train_pos)) + '\n')
#log_file1.write("length of pu_test_pos: " + str(len(pu_test_pos)) + '\n')
pu_train_neg = sc.parallelize(pu_neg_features, 50).takeSample(False, int(len(pu_neg_features) * gamma))  # list
pu_test_neg = sc.parallelize(pu_neg_features).subtractByKey(sc.parallelize(pu_train_neg)).collect()
log_file1.write("length of pu_train_neg: " + str(len(pu_train_neg)) + '\n')
log_file1.write("length of pu_test_neg: " + str(len(pu_test_neg)) + '\n')
log_file1.write(time.asctime() + '\n\n')
print(time.asctime())

pu_train_data = sc.parallelize(pu_train_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
.union(sc.parallelize(pu_train_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

Fri Feb 17 15:12:46 2017
Fri Feb 17 15:13:47 2017
Fri Feb 17 15:14:34 2017


In [10]:
pu_test_data = sc.parallelize(pu_test_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1]), pair=x[0]))\
.union(sc.parallelize(pu_test_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]), pair=x[0]))).toDF()

In [ ]:
### LR
log_file1.write("###### LogisticRegression ######\n")
publr = LogisticRegression().fit(pu_train_data)
pu_test_data_preRes_LR = publr.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

predict_to_0_LR = pu_test_data_preRes_LR.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
predict_to_0_prob_LR = predict_to_0_LR.map(lambda x: x.probability[0]).collect()

prob_LR_max = max(predict_to_0_prob_LR)
prob_LR_mean = sum(predict_to_0_prob_LR)/len(predict_to_0_prob_LR)
print(len(predict_to_0_prob_LR), prob_LR_max, min(predict_to_0_prob_LR), prob_LR_mean)
log_file1.write("len(predict_to_0_prob_LR) ++ max(predict_to_0_prob_LR) ++ min(predict_to_0_prob_LR) ++ mean(predict_to_0_prob_LR) \n")
log_file1.write(str(len(predict_to_0_prob_LR)) + ', ' + str(prob_LR_max) + ', ' + str(min(predict_to_0_prob_LR)) + ', ' + str(prob_LR_mean) + '\n')
log_file1.write(time.asctime() + '\n')

RN_LR_max = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] >= prob_LR_max).map(lambda x: x.features).collect()
RN_LR_mean = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] > prob_LR_mean).map(lambda x: x.features).collect()
log_file1.write('len(RN_LR_max): '+str(len(RN_LR_max))+'\n')
log_file1.write("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
print('len(RN_LR_max): '+str(len(RN_LR_max))+'\n')
print("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
sc.parallelize(RN_LR_max).saveAsTextFile(base_path + "RN/RN_LR_max-" + str(alfa))
sc.parallelize(RN_LR_mean).saveAsTextFile(base_path + "RN/RN_LR_mean-" + str(alfa))

In [11]:
### NB
log_file1.write("###### NaiveBayes ######\n")
pubNB = NaiveBayes().fit(pu_train_data)
pu_test_data_preRes_NB = pubNB.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

predict_to_0_NB = pu_test_data_preRes_NB.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
predict_to_0_prob_NB = predict_to_0_NB.map(lambda x: x.probability[0]).collect()

prob_NB_max = max(predict_to_0_prob_NB)
prob_NB_mean = sum(predict_to_0_prob_NB)/len(predict_to_0_prob_NB)
print(len(predict_to_0_prob_NB), prob_NB_max, min(predict_to_0_prob_NB), prob_NB_mean)
log_file1.write("len(predict_to_0_prob_NB) ++ max(predict_to_0_prob_NB) ++ min(predict_to_0_prob_NB) ++ mean(predict_to_0_prob_NB) \n")
log_file1.write(str(len(predict_to_0_prob_NB)) + ', ' + str(prob_NB_max) + ', ' + str(min(predict_to_0_prob_NB)) + ', ' + str(prob_NB_mean) + '\n')
log_file1.write(time.asctime() + '\n')

RN_NB_max = pu_test_data_preRes_NB.rdd.filter(lambda x: x.probability[0] >= prob_NB_max).map(lambda x: x.features).collect()
RN_NB_mean = pu_test_data_preRes_NB.rdd.filter(lambda x: x.probability[0] > prob_NB_mean).map(lambda x: x.features).collect()
log_file1.write('len(RN_NB_max): '+str(len(RN_NB_max))+'\n')
log_file1.write("len(RN_NB_mean):" + str(len(RN_NB_mean))+'\n')
print('len(RN_NB_max): '+str(len(RN_NB_max))+'\n')
print("len(RN_NB_mean):" + str(len(RN_NB_mean))+'\n')
sc.parallelize(RN_NB_max).saveAsTextFile(base_path + "RN/RN_NB_max-" + str(alfa))
sc.parallelize(RN_NB_mean).saveAsTextFile(base_path + "RN/RN_NB_mean-" + str(alfa))

56404 1.0 0.582044445478 0.999992552523
len(RN_NB_max): 153265

len(RN_NB_mean):155050



In [16]:
### DT
log_file1.write("###### DecisionTreeClassifier ######\n")
pubDT = DecisionTreeClassifier().fit(pu_train_data)
pu_test_data_preRes_DT = pubDT.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer") 

predict_to_0_DT = pu_test_data_preRes_DT.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
predict_to_0_prob_DT = predict_to_0_DT.map(lambda x: x.probability[0]).collect()

prob_DT_max = max(predict_to_0_prob_DT)
prob_DT_mean = sum(predict_to_0_prob_DT)/len(predict_to_0_prob_DT)
print(len(predict_to_0_prob_DT), prob_DT_max, min(predict_to_0_prob_DT), prob_DT_mean)
log_file1.write("len(predict_to_0_prob_DT) ++ max(predict_to_0_prob_DT) ++ min(predict_to_0_prob_DT) ++ mean(predict_to_0_prob_DT) \n")
log_file1.write(str(len(predict_to_0_prob_DT)) + ', ' + str(prob_DT_max) + ', ' + str(min(predict_to_0_prob_DT)) + ', ' + str(prob_DT_mean) + '\n')
log_file1.write(time.asctime() + '\n')

RN_DT_max = pu_test_data_preRes_DT.rdd.filter(lambda x: x.probability[0] >= prob_DT_max).map(lambda x: x.features).collect()
RN_DT_mean = pu_test_data_preRes_DT.rdd.filter(lambda x: x.probability[0] > prob_DT_mean).map(lambda x: x.features).collect()
log_file1.write('len(RN_DT_max): '+str(len(RN_DT_max))+'\n')
log_file1.write("len(RN_DT_mean):" + str(len(RN_DT_mean))+'\n')
print('len(RN_DT_max): '+str(len(RN_DT_max))+'\n')
print("len(RN_DT_mean):" + str(len(RN_DT_mean))+'\n')
sc.parallelize(RN_DT_max).saveAsTextFile(base_path + "RN/RN_DT_max-" + str(alfa))
sc.parallelize(RN_DT_mean).saveAsTextFile(base_path + "RN/RN_DT_mean-" + str(alfa))

### RandomForestClassifier
log_file1.write("###### RandomForestClassifier ######\n")
pubRF = RandomForestClassifier().fit(pu_train_data)
pu_test_data_preRes_RF = pubRF.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

predict_to_0_RF = pu_test_data_preRes_RF.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
predict_to_0_prob_RF = predict_to_0_RF.map(lambda x: x.probability[0]).collect()

prob_RF_max = max(predict_to_0_prob_RF)
prob_RF_mean = sum(predict_to_0_prob_RF)/len(predict_to_0_prob_RF)
print(len(predict_to_0_prob_RF), prob_RF_max, min(predict_to_0_prob_RF), prob_RF_mean)
log_file1.write("len(predict_to_0_prob_RF) ++ max(predict_to_0_prob_RF) ++ min(predict_to_0_prob_RF) ++ mean(predict_to_0_prob_RF) \n")
log_file1.write(str(len(predict_to_0_prob_RF)) + ', ' + str(prob_RF_max) + ', ' + str(min(predict_to_0_prob_RF)) + ', ' + str(prob_RF_mean) + '\n')
log_file1.write(time.asctime() + '\n')

RN_RF_max = pu_test_data_preRes_RF.rdd.filter(lambda x: x.probability[0] >= prob_RF_max).map(lambda x: x.features).collect()
RN_RF_mean = pu_test_data_preRes_RF.rdd.filter(lambda x: x.probability[0] > prob_RF_mean).map(lambda x: x.features).collect()
log_file1.write('len(RN_RF_max): '+str(len(RN_RF_max))+'\n')
log_file1.write("len(RN_RF_mean):" + str(len(RN_RF_mean))+'\n')
print('len(RN_RF_max): '+str(len(RN_RF_max))+'\n')
print("len(RN_RF_mean):" + str(len(RN_RF_mean))+'\n')
sc.parallelize(RN_RF_max).saveAsTextFile(base_path + "RN/RN_RF_max-" + str(alfa))
sc.parallelize(RN_RF_mean).saveAsTextFile(base_path + "RN/RN_RF_mean-" + str(alfa))

18193 0.999929397229 0.506172839506 0.625787355222
len(RN_DT_max): 265276

len(RN_DT_mean):289695

21187 0.990834485173 0.500017124999 0.614413289861
len(RN_RF_max): 94118

len(RN_RF_mean):291137



In [17]:
log_file1.close()

In [18]:
log_file2 = open('qq_PU_alfa_2_v3.log', 'a+')
log_file2.write('\n' + time.asctime() + '\n')
log_file2.write("===Predict with RN===\n" + '\n')
log_file2.write( "==========alfa: " + str(alfa) + "============\n")
print(time.asctime())

Fri Feb 17 15:26:20 2017


In [19]:
# NB
log_file2.write('###### NaiveBayes #######\n')
RN_NB = sc.textFile(base_path + "RN/RN_NB_mean-" + str(alfa)).map(lambda x: eval(x))

PU_train_data_NB = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
.union(RN_NB.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense

NB = NaiveBayes().fit(PU_train_data_NB)
predict_4th_day_NB = NB.transform(test_data_4thday.select('features'))

y_predict_PU_NB = [i.prediction for i in predict_4th_day_NB.select('prediction').collect()]

recall_score_PU_NB = recall_score(y_label, y_predict_PU_NB)
precision_score_PU_NB = precision_score(y_label, y_predict_PU_NB)
f1_score_PU_NB = f1_score(y_label, y_predict_PU_NB)
tp_NB = 0
tn_NB = 0
for i in range(500000):
    if y_label[i] == 1.0 and y_predict_PU_NB[i] == 1.0:
        tp_NB += 1
    if y_label[i] == 0.0 and y_predict_PU_NB[i] == 0.0:
        tn_NB += 1
accuracy_NB = (tp_NB + tn_NB)/500000

print(recall_score_PU_NB, precision_score_PU_NB, f1_score_PU_NB, accuracy_NB)

log_file2.write('\n')
log_file2.write("recall_score_PU_NB: " + str(recall_score_PU_NB) + '\n')
log_file2.write("precision_score_PU_NB: " + str(precision_score_PU_NB) + '\n')
log_file2.write("f1_score_PU_NB: " + str(f1_score_PU_NB) + '\n')
log_file2.write("accuracy_NB: " + str(accuracy_NB) + '\n')
log_file2.write(time.asctime() + '\n')
log_file2.write('\n')

print(time.asctime())

# DT
log_file2.write('###### DecisionTreeClassifier #######\n')
RN_DT = sc.textFile(base_path + "RN/RN_DT_mean-" + str(alfa)).map(lambda x: eval(x))

PU_train_data_DT = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
.union(RN_DT.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense

DT = DecisionTreeClassifier().fit(PU_train_data_DT)
predict_4th_day_DT = DT.transform(test_data_4thday.select('features'))

y_predict_PU_DT = [i.prediction for i in predict_4th_day_DT.select('prediction').collect()]

recall_score_PU_DT = recall_score(y_label, y_predict_PU_DT)
precision_score_PU_DT = precision_score(y_label, y_predict_PU_DT)
f1_score_PU_DT = f1_score(y_label, y_predict_PU_DT)
tp_DT = 0
tn_DT = 0
for i in range(500000):
    if y_label[i] == 1.0 and y_predict_PU_DT[i] == 1.0:
        tp_DT += 1
    if y_label[i] == 0.0 and y_predict_PU_DT[i] == 0.0:
        tn_DT += 1
accuracy_DT = (tp_DT + tn_DT)/500000

print(recall_score_PU_DT, precision_score_PU_DT, f1_score_PU_DT, accuracy_DT)

log_file2.write('\n')
log_file2.write("recall_score_PU_DT: " + str(recall_score_PU_DT) + '\n')
log_file2.write("precision_score_PU_DT: " + str(precision_score_PU_DT) + '\n')
log_file2.write("f1_score_PU_DT: " + str(f1_score_PU_DT) + '\n')
log_file2.write("accuracy_DT: " + str(accuracy_DT) + '\n')
log_file2.write(time.asctime() + '\n')
log_file2.write('\n')

print(time.asctime())

# RF
log_file2.write('###### RandomForestClassifier #######\n')
RN_RF = sc.textFile(base_path + "RN/RN_RF_mean-" + str(alfa)).map(lambda x: eval(x))

PU_train_data_RF = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
.union(RN_RF.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense

RF = RandomForestClassifier().fit(PU_train_data_RF)
predict_4th_day_RF = RF.transform(test_data_4thday.select('features'))

y_predict_PU_RF = [i.prediction for i in predict_4th_day_RF.select('prediction').collect()]

recall_score_PU_RF = recall_score(y_label, y_predict_PU_RF)
precision_score_PU_RF = precision_score(y_label, y_predict_PU_RF)
f1_score_PU_RF = f1_score(y_label, y_predict_PU_RF)
tp_RF = 0
tn_RF = 0
for i in range(500000):
    if y_label[i] == 1.0 and y_predict_PU_RF[i] == 1.0:
        tp_RF += 1
    if y_label[i] == 0.0 and y_predict_PU_RF[i] == 0.0:
        tn_RF += 1
accuracy_RF = (tp_RF + tn_RF)/500000

print(recall_score_PU_RF, precision_score_PU_RF, f1_score_PU_RF, accuracy_RF)

log_file2.write('\n')
log_file2.write("recall_score_PU_RF: " + str(recall_score_PU_RF) + '\n')
log_file2.write("precision_score_PU_RF: " + str(precision_score_PU_RF) + '\n')
log_file2.write("f1_score_PU_RF: " + str(f1_score_PU_RF) + '\n')
log_file2.write("accuracy_RF: " + str(accuracy_RF) + '\n')
log_file2.write(time.asctime() + '\n')
log_file2.write('\n')

print(time.asctime())

0.768725361367 0.00723959128279 0.01434409481 0.03524
Fri Feb 17 15:30:52 2017
0.993648707841 0.010038876584 0.0198769356135 0.10513
Fri Feb 17 15:37:15 2017
0.916119141481 0.0638869797633 0.119444325467 0.87665
Fri Feb 17 15:43:23 2017


In [20]:
log_file2.close()

In [37]:
alfa = 0.05
train_negative_pairs = sc.textFile(base_path + "wbb_train_negative_pairs-" + str(alfa)).map(lambda x: eval(x)).collect()

#log_file1.write("length of train_negative_pairs: " + str(len(train_negative_pairs)) + '\n')
print(time.asctime())
#log_file1.write(time.asctime() + '\n')

# ===============================================
# with PU
blta = 0.8  # 从train pos 里面取一部分加到 neg 里面组成PU数据
pnsplit = int(train_positive_pairs.count()*blta)   # pnsplit以前的部分作为pu_pos, 以后的部分作为pu_neg
pu_pos_features = train_positive_pairs.collect()[:pnsplit]
pu_neg_features = train_negative_pairs + train_positive_pairs.collect()[pnsplit:]

#log_file1.write("length of pu_pos_features: " + str(len(pu_pos_features)) + '\n')
#log_file1.write("length of pu_neg_features: " + str(len(pu_neg_features)) + '\n')
#log_file1.write(time.asctime() + '\n\n')
print(time.asctime())

gamma=0.3  # 从PU数据里面取一部分训练，另一部分作为测试
pu_train_pos = sc.parallelize(pu_pos_features).takeSample(False, int(len(pu_pos_features) * gamma))  # list
pu_test_pos = sc.parallelize(pu_pos_features).subtractByKey(sc.parallelize(pu_train_pos)).collect()
#log_file1.write("length of pu_train_pos: " + str(len(pu_train_pos)) + '\n')
#log_file1.write("length of pu_test_pos: " + str(len(pu_test_pos)) + '\n')
pu_train_neg = sc.parallelize(pu_neg_features, 50).takeSample(False, int(len(pu_neg_features) * gamma))  # list
pu_test_neg = sc.parallelize(pu_neg_features).subtractByKey(sc.parallelize(pu_train_neg)).collect()
#log_file1.write("length of pu_train_neg: " + str(len(pu_train_neg)) + '\n')
#log_file1.write("length of pu_test_neg: " + str(len(pu_test_neg)) + '\n')
#log_file1.write(time.asctime() + '\n\n')
print(time.asctime())

pu_train_data = sc.parallelize(pu_train_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
.union(sc.parallelize(pu_train_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

pu_test_data = sc.parallelize(pu_test_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
.union(sc.parallelize(pu_test_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

Fri Feb 17 11:22:56 2017
Fri Feb 17 11:24:06 2017
Fri Feb 17 11:24:52 2017


In [38]:
pu_test_data = sc.parallelize(pu_test_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1]), pair=x[0]))\
.union(sc.parallelize(pu_test_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]), pair=x[0]))).toDF()

In [13]:
pu_test_data.count()

454320

In [10]:
pu_test_data.show(2)

+--------------------+-----+--------------------+
|            features|label|                pair|
+--------------------+-----+--------------------+
|[0.0,0.0,0.0,0.0,...|  1.0|day1-861381339898...|
|[0.0,0.0,0.0,0.0,...|  1.0|day3-861385194265...|
+--------------------+-----+--------------------+
only showing top 2 rows



In [39]:
print(time.asctime())
pu_spy = [Vectors.dense(x[1]) for x in train_positive_pairs.collect()[pnsplit:]]
publr = LogisticRegression().fit(pu_train_data)
pu_test_data_preRes_LR = publr.transform(pu_test_data.select("features")) 

print(time.asctime())

Fri Feb 17 11:25:04 2017
Fri Feb 17 11:25:56 2017


In [7]:
len(pu_spy)

47118

In [12]:
pu_test_data_preRes_LR.show(2)

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[0.0,0.0,0.0,0.0,...|[0.80226282901117...|[0.69045831399825...|       0.0|
|[0.0,0.0,0.0,0.0,...|[-40.936007754644...|[1.66616390165119...|       1.0|
+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



In [27]:
pu_test_data_preRes_LR.count()

454320

In [40]:
aaaaa = pu_test_data_preRes_LR.join(pu_test_data, 'features', "left_outer")

In [29]:
aaaaa.count()

454460

In [16]:
aaaaa.show(5)

+--------------------+--------------------+--------------------+----------+-----+--------------------+
|            features|       rawPrediction|         probability|prediction|label|                pair|
+--------------------+--------------------+--------------------+----------+-----+--------------------+
|[0.0,0.0,0.0,0.0,...|[0.44888708795668...|[0.61037459658138...|       0.0|  0.0|day2-861395169813...|
|[0.0,0.0,0.0,0.0,...|[3.06571481451773...|[0.95545615175954...|       0.0|  0.0|day123-8613951946...|
|[0.0,0.0,0.0,0.0,...|[2.39139678017373...|[0.91616890800357...|       0.0|  0.0|day123-8613951680...|
|[0.0,0.0,0.0,0.0,...|[2.60683271638322...|[0.93130002918038...|       0.0|  0.0|day123-8613951708...|
|[0.0,0.0,0.0,0.0,...|[0.60560027801470...|[0.64693651394611...|       0.0|  0.0|day2-861392143050...|
+--------------------+--------------------+--------------------+----------+-----+--------------------+
only showing top 5 rows



In [7]:
print(pu_test_data_preRes_LR.rdd.take(10))

[Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 4.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 25.0, 85.0, 100.0, 79109.0, 1037.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7725.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7725.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 156.0, 147.0, 180.0, 142.0, 99.0, 117.0, 117.0, 203.0, 333.0, 349.0, 413.0, 453.0, 455.0, 463.0, 502.0, 407.0, 506.0, 405.0,

In [9]:
xxx = [Vectors.dense([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 4.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 25.0, 85.0, 100.0, 79109.0, 1037.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7725.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7725.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 156.0, 147.0, 180.0, 142.0, 99.0, 117.0, 117.0, 203.0, 333.0, 349.0, 413.0, 453.0, 455.0, 463.0, 502.0, 407.0, 506.0, 405.0, 432.0, 472.0, 460.0, 375.0, 353.0, 260.0, 7799.0, 43548.0, 31033.0, 54799406.0]),Vectors.dense([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 41.0, 38.0, 10058.0, 1037.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7725.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7725.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 156.0, 147.0, 180.0, 142.0, 99.0, 117.0, 117.0, 203.0, 333.0, 349.0, 413.0, 453.0, 455.0, 463.0, 502.0, 407.0, 506.0, 405.0, 432.0, 472.0, 460.0, 375.0, 353.0, 260.0, 7799.0, 43548.0, 31033.0, 54799406.0]), Vectors.dense([0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 17.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 23.0, 0.0, 0.0, 0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 26.0, 72.0, 1283.0, 902.0, 409932.0, 743.0, 0.0, 33.0, 0.0, 0.0, 0.0, 2.0, 123.0, 0.0, 5980.0, 0.0, 0.0, 2.0, 195.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 185.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 111.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 0.0, 5980.0, 33.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 306.0, 150.0, 58.0, 81.0, 70.0, 53.0, 168.0, 307.0, 284.0, 245.0, 267.0, 168.0, 335.0, 207.0, 278.0, 241.0, 268.0, 278.0, 344.0, 357.0, 397.0, 396.0, 465.0, 612.0, 6335.0, 57860.0, 53226.0, 46377859.0])]

In [41]:
def pos_predict_to_0(x):
    if x.pair.split('-')[0] != 'day123' and x.prediction == 0.0:
        return x

In [42]:
print(time.asctime())
predict_to_0_LR = aaaaa.rdd.filter(lambda x: pos_predict_to_0(x))#(x.features in xxx and x.prediction == 0.0))

Fri Feb 17 11:25:56 2017


In [19]:
predict_to_0_LR.take(10)

[Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 5.0, 6.0, 32.0, 34.0, 28751.0, 997.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2510.0, 0.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2510.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 97.0, 66.0, 58.0, 21.0, 29.0, 35.0, 39.0, 104.0, 101.0, 131.0, 128.0, 143.0, 100.0, 121.0, 112.0, 123.0, 120.0, 134.0, 128.0, 1

In [43]:
predict_to_0_prob_LR = predict_to_0_LR.map(lambda x: x.probability[0]).collect()

In [45]:
#predict_to_0_prob_LR

In [44]:
prob_LR_max = max(predict_to_0_prob_LR)
prob_LR_mean = sum(predict_to_0_prob_LR)/len(predict_to_0_prob_LR)
print(len(predict_to_0_prob_LR), prob_LR_max, min(predict_to_0_prob_LR), prob_LR_mean)

63587 0.999993652141 0.500028466335 0.741877658985


In [46]:
RN_LR_max = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] >= prob_LR_max).map(lambda x: x.features).collect()
RN_LR_mean = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] > prob_LR_mean).map(lambda x: x.features).collect()


In [47]:
print(len(RN_LR_max), len(RN_LR_mean))
print(time.asctime())

244 299914
Fri Feb 17 11:28:07 2017


In [48]:
PU_train_data_LR = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
.union(sc.parallelize(RN_LR_mean).map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense

lr = LogisticRegression().fit(PU_train_data_LR)
predict_4th_day_LR = lr.transform(test_data_4thday.select('features'))

y_predict_PU_LR = [i.prediction for i in predict_4th_day_LR.select('prediction').collect()]

recall_score_PU_LR = recall_score(y_label, y_predict_PU_LR)
precision_score_PU_LR = precision_score(y_label, y_predict_PU_LR)
f1_score_PU_LR = f1_score(y_label, y_predict_PU_LR)
tp_LR = 0
tn_LR = 0
for i in range(500000):
    if y_label[i] == 1.0 and y_predict_PU_LR[i] == 1.0:
        tp_LR += 1
    if y_label[i] == 0.0 and y_predict_PU_LR[i] == 0.0:
        tn_LR += 1
accuracy_LR = (tp_LR + tn_LR)/500000

print(time.asctime())
print(recall_score_PU_LR, precision_score_PU_LR, f1_score_PU_LR, accuracy_LR)

Fri Feb 17 11:33:48 2017
0.877792378449 0.0635313138999 0.118486985056 0.880726


In [7]:

predict_to_0_prob_LR = predict_to_0_LR.map(lambda x: x.probability[0]).collect()
"""predict_to_0_prob_LR = []
for i in pu_test_data_preRes_LR.collect():
    if i.prediction == 0.0 and i.features in pu_spy:
        predict_to_0_prob_LR.append(i.probability[0])"""
prob_LR_max = max(predict_to_0_prob_LR)
prob_LR_mean = sum(predict_to_0_prob_LR)/len(predict_to_0_prob_LR)
print(len(predict_to_0_prob_LR), prob_LR_max, min(predict_to_0_prob_LR), prob_LR_mean)
#RN_LR_max = []
#RN_LR_mean = []
RN_LR_max = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] >= prob_LR_max).map(lambda x: x.features).collect()
RN_LR_mean = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] > prob_LR_mean).map(lambda x: x.features).collect()

"""for i in pu_test_data_preRes_LR.collect():
    if i.probability[0] >= prob_LR_max:
        RN_LR_max.append(i.features)
    if i.probability[0] > prob_LR_mean:
        RN_LR_mean.append(i.features)"""
print(len(RN_LR_max), len(RN_LR_mean))
print(time.asctime())

Thu Feb 16 21:41:55 2017


KeyboardInterrupt: 

In [ ]:
print(RN_LR_max[:3])

In [25]:
type(pu_test_data_preRes_LR)

pyspark.sql.dataframe.DataFrame

In [7]:
with open('qq_PU_alfa_1_v2.log', 'a+') as log_file1:
    # 
    for alfa in [0.05]:# 0.10, 0.15, 0.20, 0.25, 0.30, 0.35]:
        log_file1.write(time.asctime() + '\n')
        log_file1.write("===GET RN===\n" + '\n')
        log_file1.write( "==========alfa: " + str(alfa) + "============\n")
        print(time.asctime())
        # train_unlabeled -> train_neg
        #train_negative_pairs = train_unlabled_pairs.takeSample(False, int(alfa * 8268837))
        #sc.parallelize(train_negative_pairs).coalesce(20).saveAsTextFile(base_path + "wbb_train_negative_pairs-" + str(alfa))
        train_negative_pairs = sc.textFile(base_path + "wbb_train_negative_pairs-" + str(alfa)).map(lambda x: eval(x)).collect()

        log_file1.write("length of train_negative_pairs: " + str(len(train_negative_pairs)) + '\n')
        print(time.asctime())
        log_file1.write(time.asctime() + '\n')

        # ===============================================
        # with PU
        blta = 0.8  # 从train pos 里面取一部分加到 neg 里面组成PU数据
        pnsplit = int(train_positive_pairs.count()*blta)   # pnsplit以前的部分作为pu_pos, 以后的部分作为pu_neg
        pu_pos_features = train_positive_pairs.collect()[:pnsplit]
        pu_neg_features = train_negative_pairs + train_positive_pairs.collect()[pnsplit:]
        
        log_file1.write("length of pu_pos_features: " + str(len(pu_pos_features)) + '\n')
        log_file1.write("length of pu_neg_features: " + str(len(pu_neg_features)) + '\n')
        log_file1.write(time.asctime() + '\n\n')
        print(time.asctime())

        gamma=0.3  # 从PU数据里面取一部分训练，另一部分作为测试
        pu_train_pos = sc.parallelize(pu_pos_features).takeSample(False, int(len(pu_pos_features) * gamma))  # list
        pu_test_pos = sc.parallelize(pu_pos_features).subtractByKey(sc.parallelize(pu_train_pos)).collect()
        log_file1.write("length of pu_train_pos: " + str(len(pu_train_pos)) + '\n')
        log_file1.write("length of pu_test_pos: " + str(len(pu_test_pos)) + '\n')
        pu_train_neg = sc.parallelize(pu_neg_features, 50).takeSample(False, int(len(pu_neg_features) * gamma))  # list
        pu_test_neg = sc.parallelize(pu_neg_features).subtractByKey(sc.parallelize(pu_train_neg)).collect()
        log_file1.write("length of pu_train_neg: " + str(len(pu_train_neg)) + '\n')
        log_file1.write("length of pu_test_neg: " + str(len(pu_test_neg)) + '\n')
        log_file1.write(time.asctime() + '\n\n')
        print(time.asctime())

        pu_train_data = sc.parallelize(pu_train_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(sc.parallelize(pu_train_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

        pu_test_data = sc.parallelize(pu_test_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(sc.parallelize(pu_test_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()
        
        pu_test_data_list = pu_test_data.collect()
        length_pu_test_data = len(pu_test_data)

        ### LR
        publr = LogisticRegression().fit(pu_train_data)
        pu_test_data_preRes_LR = publr.transform(pu_test_data.select("features")) #.join(pu_test_data, 'features')

        pu_test_data_preRes_LR_temp = pu_test_data_preRes_LR.filter((col('prediction') == 0) & (col('label') == 0))\
        .select('features', 'probability').collect()

        predict_to_0_prob_LR = []
        for i in pu_test_data_preRes_LR_temp:
            if i.features[-1] != 0 and i.features[-2] != 0 and i.features[-3] != 0:
                predict_to_0_prob_LR.append(i.probability[0])
        prob_LR_max = max(predict_to_0_prob_LR)
        prob_LR_mean = sum(predict_to_0_prob_LR)/len(predict_to_0_prob_LR)
        print(len(predict_to_0_prob_LR), prob_LR_max, min(predict_to_0_prob_LR), prob_LR_mean)
        log_file1.write("len(predict_to_0_prob_LR) ++ max(predict_to_0_prob_LR) ++ min(predict_to_0_prob_LR) ++ mean(predict_to_0_prob_LR) \n")
        log_file1.write(str(len(predict_to_0_prob_LR)) + ', ' + str(prob_LR_max) + ', ' + str(min(predict_to_0_prob_LR)) + ', ' + str(prob_LR_mean) + '\n')
        log_file1.write(time.asctime() + '\n')
        
        bbb_LR = pu_test_data_preRes_LR.filter(pu_test_data_preRes_LR.label == 0).select('features', 'probability').collect()
        RN_LR_max = []
        RN_LR_mean = []
        for i in bbb_LR:
            if i.probability[0] >= prob_LR_max:
                RN_LR_max.append(i.features)
            if i.probability[0] > prob_LR_mean:
                RN_LR_mean.append(i.features)

        log_file1.write('len(RN_LR_max): '+str(len(RN_LR_max))+'\n')
        log_file1.write("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
        print('len(RN_LR_max): '+str(len(RN_LR_max))+'\n)
        print("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
        sc.parallelize(RN_LR_max).saveAsTextFile(base_path + "RN/RN_LR_max-" + str(alfa))
        sc.parallelize(RN_LR_mean).saveAsTextFile(base_path + "RN/RN_LR_mean-" + str(alfa))
        
        ##############################
        # NB
        pubNB = NaiveBayes().fit(pu_train_data)
        pu_test_data_preRes_NB = pubNB.transform(pu_test_data.select("features")).join(pu_test_data, 'features')

        pu_test_data_preRes_NB_temp = pu_test_data_preRes_NB.filter((col('prediction') == 0) & (col('label') == 0))\
        .select('features', 'probability').collect()

        predict_to_0_prob_NB = []
        for i in pu_test_data_preRes_NB_temp:
            if i.features[-1] != 0 and i.features[-2] != 0 and i.features[-3] != 0:
                predict_to_0_prob_NB.append(i.probability[0])
        prob_NB_max = max(predict_to_0_prob_NB)
        prob_NB_mean = sum(predict_to_0_prob_NB)/len(predict_to_0_prob_NB)
        print(len(predict_to_0_prob_NB), prob_NB_max, min(predict_to_0_prob_NB), prob_NB_mean)
        log_file1.write("len(predict_to_0_prob_NB) ++ max(predict_to_0_prob_NB) ++ min(predict_to_0_prob_NB) ++ mean(predict_to_0_prob_NB) \n")
        log_file1.write(str(len(predict_to_0_prob_NB)) + ', ' + str(prob_NB_max) + ', ' + str(min(predict_to_0_prob_NB)) + ', ' + str(prob_NB_mean) + '\n')
        log_file1.write(time.asctime() + '\n')
        
        bbb_NB = pu_test_data_preRes_NB.filter(pu_test_data_preRes_NB.label == 0).select('features', 'probability').collect()
        RN_NB_max = []
        RN_NB_mean = []
        for i in bbb_NB:
            if i.probability[0] >= prob_NB_max:
                RN_NB_max.append(i.features)
            if i.probability[0] > prob_NB_mean:
                RN_NB_mean.append(i.features)

        log_file1.write("len(RN_NB_max): "+str(len(RN_NB_max))+'\n')
        log_file1.write("len(RN_NB_mean): "+str(len(RN_NB_mean))+'\n')
        log_file1.write(time.asctime() + '\n')
        print('len(RN_NB_max): '+str(len(RN_NB_max))+'\n)
        print("len(RN_NB_mean):" + str(len(RN_NB_mean))+'\n')
        sc.parallelize(RN_NB_max).saveAsTextFile(base_path + "RN/RN_NB_max-" + str(alfa))
        sc.parallelize(RN_NB_mean).saveAsTextFile(base_path + "RN/RN_NB_mean-" + str(alfa))
        
        ###############################
        # DT
        pubDT = DecisionTreeClassifier().fit(pu_train_data)
        pu_test_data_preRes_DT = pubDT.transform(pu_test_data.select("features")).join(pu_test_data, 'features')

        pu_test_data_preRes_DT_temp = pu_test_data_preRes_DT.filter((col('prediction') == 0) & (col('label') == 0))\
        .select('features', 'probability').collect()

        predict_to_0_prob_DT = []
        for i in pu_test_data_preRes_DT_temp:
            if i.features[-1] != 0 and i.features[-2] != 0 and i.features[-3] != 0:
                predict_to_0_prob_DT.append(i.probability[0])
        prob_DT_max = max(predict_to_0_prob_DT)
        prob_DT_mean = sum(predict_to_0_prob_DT)/len(predict_to_0_prob_DT)
        print(len(predict_to_0_prob_DT), prob_DT_max, min(predict_to_0_prob_DT), prob_DT_mean)
        log_file1.write("len(predict_to_0_prob_DT) ++ max(predict_to_0_prob_DT) ++ min(predict_to_0_prob_DT) ++ mean(predict_to_0_prob_DT) \n")
        log_file1.write(str(len(predict_to_0_prob_DT)) + ', ' + str(prob_DT_max) + ', ' + str(min(predict_to_0_prob_DT)) + ', ' + str(prob_DT_mean) + '\n')
        log_file1.write(time.asctime() + '\n')
        
        bbb_DT = pu_test_data_preRes_DT.filter(pu_test_data_preRes_DT.label == 0).select('features', 'probability').collect()
        RN_DT_max = []
        RN_DT_mean = []
        for i in bbb_DT:
            if i.probability[0] >= prob_DT_max:
                RN_DT_max.append(i.features)
            if i.probability[0] > prob_DT_mean:
                RN_DT_mean.append(i.features)

        log_file1.write("len(RN_DT_max): "+str(len(RN_DT_max))+'\n')
        log_file1.write("len(RN_DT_max): "+str(len(RN_DT_mean))+'\n')
        log_file1.write(time.asctime() + '\n')
        print('len(RN_DT_max): '+str(len(RN_DT_max))+'\n)
        print("len(RN_DT_mean):" + str(len(RN_DT_mean))+'\n')
        sc.parallelize(RN_DT_max).saveAsTextFile(base_path + "RN/RN_DT_max-" + str(alfa))
        sc.parallelize(RN_DT_mean).saveAsTextFile(base_path + "RN/RN_DT_mean-" + str(alfa))

Wed Feb 15 18:02:04 2017
Wed Feb 15 18:02:24 2017
Wed Feb 15 18:03:22 2017
293895 1.0 0.500005465819 0.893220571531
118258 1.0 0.506136879818 0.999646331541
295498 0.999885225928 0.588679245283 0.980899639987


In [8]:
with open('qq_PU_alfa_2.log', 'a+') as log_file2:
    for alfa in [0.05]:# 0.10, 0.15, 0.20, 0.25, 0.30, 0.35]:
        log_file2.write('\n' + time.asctime() + '\n')
        log_file2.write("===Predict with RN===\n" + '\n')
        log_file2.write( "==========alfa: " + str(alfa) + "============\n")
        print(time.asctime())
        
        # LR
        RN_LR = sc.textFile(base_path + "RN/RN_LR_mean-" + str(alfa)).map(lambda x: eval(x))
        
        PU_train_data_LR = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_LR.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        lr = LogisticRegression().fit(PU_train_data_LR)
        predict_4th_day_LR = lr.transform(test_data_4thday.select('features')).join(test_data_4thday, 'features')
        
        y_label_PU_LR = [i.label for i in predict_4th_day_LR.select('label').collect()]
        y_predict_PU_LR = [i.prediction for i in predict_4th_day_LR.select('prediction').collect()]
        
        recall_score_PU_LR = recall_score(y_label_PU_LR, y_predict_PU_LR)
        precision_score_PU_LR = precision_score(y_label_PU_LR, y_predict_PU_LR)
        f1_score_PU_LR = f1_score(y_label_PU_LR, y_predict_PU_LR)
        
        print(recall_score_PU_LR, precision_score_PU_LR, f1_score_PU_LR)

        log_file2.write('\n')
        log_file2.write("recall_score_PU_LR: " + str(recall_score_PU_LR) + '\n')
        log_file2.write("precision_score_PU_LR: " + str(precision_score_PU_LR) + '\n')
        log_file2.write("f1_score_PU_LR: " + str(f1_score_PU_LR) + '\n')
        log_file2.write(time.asctime() + '\n')
        log_file2.write('\n')

        print(time.asctime())
        
        # NB
        RN_NB = sc.textFile(base_path + "RN/RN_NB_mean-" + str(alfa)).map(lambda x: eval(x))
        
        PU_train_data_NB = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_NB.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()
        
        NB = NaiveBayes().fit(PU_train_data_NB)
        predict_4th_day_NB = NB.transform(test_data_4thday.select('features')).join(test_data_4thday, 'features')
        
        y_label_PU_NB = [i.label for i in predict_4th_day_NB.select('label').collect()]
        y_predict_PU_NB = [i.prediction for i in predict_4th_day_NB.select('prediction').collect()]
        
        recall_score_PU_NB = recall_score(y_label_PU_NB, y_predict_PU_NB)
        precision_score_PU_NB = precision_score(y_label_PU_NB, y_predict_PU_NB)
        f1_score_PU_NB = f1_score(y_label_PU_NB, y_predict_PU_NB)
        
        print(recall_score_PU_NB, precision_score_PU_NB, f1_score_PU_NB)

        log_file2.write('\n')
        log_file2.write("recall_score_PU_NB: " + str(recall_score_PU_NB) + '\n')
        log_file2.write("precision_score_PU_NB: " + str(precision_score_PU_NB) + '\n')
        log_file2.write("f1_score_PU_NB: " + str(f1_score_PU_NB) + '\n')
        log_file2.write(time.asctime() + '\n')
        log_file2.write('\n')

        print(time.asctime())
        
        # DT
        RN_DT = sc.textFile(base_path + "RN/RN_DT_mean-" + str(alfa)).map(lambda x: eval(x))
        
        PU_train_data_DT = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_DT.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()
        
        DT = NaiveBayes().fit(PU_train_data_DT)
        predict_4th_day_DT = DT.transform(test_data_4thday.select('features')).join(test_data_4thday, 'features')
        
        y_label_PU_DT = [i.label for i in predict_4th_day_DT.select('label').collect()]
        y_predict_PU_DT = [i.prediction for i in predict_4th_day_DT.select('prediction').collect()]
        
        recall_score_PU_DT = recall_score(y_label_PU_DT, y_predict_PU_DT)
        precision_score_PU_DT = precision_score(y_label_PU_DT, y_predict_PU_DT)
        f1_score_PU_DT = f1_score(y_label_PU_DT, y_predict_PU_DT)
        
        print(recall_score_PU_DT, precision_score_PU_DT, f1_score_PU_DT)

        log_file2.write('\n')
        log_file2.write("recall_score_PU_DT: " + str(recall_score_PU_DT) + '\n')
        log_file2.write("precision_score_PU_DT: " + str(precision_score_PU_DT) + '\n')
        log_file2.write("f1_score_PU_DT: " + str(f1_score_PU_DT) + '\n')
        log_file2.write(time.asctime() + '\n')
        log_file2.write('\n')

        print(time.asctime())
        

Wed Feb 15 18:40:15 2017
Wed Feb 15 19:39:03 2017
Wed Feb 15 20:37:47 2017
Wed Feb 15 21:39:21 2017


In [9]:
predict_4th_day_LR_xx = lr.transform(test_data_4thday.select('features')).join(test_data_4thday, 'features')

In [1]:
# y_label_xx = [i.label for i in predict_4th_day_LR_xx.select('label').collect()]

In [ ]:
y_predict_xx = [i.prediction for i in predict_4th_day_LR_xx.select('prediction').collect()]

In [ ]:
recall_score_PU_LR_xx = recall_score(y_label_xx, y_predict_xx)
precision_score_PU_LR_xx = precision_score(y_label_xx, y_predict_xx)
f1_score_PU_LR_xx = f1_score(y_label_xx, y_predict_xx)

In [ ]:
print(recall_score_PU_LR_xx, precision_score_PU_LR_xx, f1_score_PU_LR_xx)

In [16]:
xxx = [Vectors.dense([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 27.0, 35.0, 29.0, 5.0, 3.0, 12.0, 5.0, 5.0, 1.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 2.0, 143.0, 740.0, 854.0, -7375.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 83.0, 123.0, 18094.0]), Vectors.dense([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 6.0, 8.0, 2.0, 1.0, 0.0, 1.0, 23.0, 115.0, 114.0, -2767.0, 291.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 577.0, 0.0, 716.0, 0.0, 0.0, 0.0, 84.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 276.0, 0.0, 0.0, 0.0, 0.0, 0.0, 300.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 83.0, 0.0, 716.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 15.0, 9.0, 8.0, 0.0, 14.0, 7.0, 23.0, 56.0, 145.0, 70.0, 92.0, 86.0, 80.0, 45.0, 31.0, 51.0, 53.0, 75.0, 126.0, 100.0, 75.0, 90.0, 99.0, 1378.0, 13684.0, 15854.0, 9779725.0])]

In [18]:
sc.parallelize(xxx).saveAsTextFile(base_path + "xxx")

In [23]:
xx = sc.textFile(base_path + "xxx").map(lambda x: Vectors.dense(eval(x)))

In [25]:
print(xx.take(2))

[DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 27.0, 35.0, 29.0, 5.0, 3.0, 12.0, 5.0, 5.0, 1.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 2.0, 143.0, 740.0, 854.0, -7375.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 83.0, 123.0, 180

In [6]:
a = [('aa',[0,1,2,3,0]), ('aa',[0,1,2,4,0]), ('aasas',[10,1,1,0,0])]
c = [('abs',[0,0,0,0,0]), ('aas',[0,1,1,0,0]), ('aaas',[110,1111,1,0,0])]

In [7]:
b = sc.parallelize(a).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]))).union((sc.parallelize(c).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1]))))).toDF()

In [8]:
b.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,1.0,2.0,3.0,...|  0.0|
|[0.0,1.0,2.0,4.0,...|  0.0|
|[10.0,1.0,1.0,0.0...|  0.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,1.0,1.0,0.0,...|  1.0|
|[110.0,1111.0,1.0...|  1.0|
+--------------------+-----+



In [9]:
xx = [i.label for i in b.select('label').collect()]

In [10]:
xx

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0]

In [12]:
xxsx =[i.features for i in b.select('features').collect()]

[DenseVector([0.0, 1.0, 2.0, 3.0, 0.0]),
 DenseVector([0.0, 1.0, 2.0, 4.0, 0.0]),
 DenseVector([10.0, 1.0, 1.0, 0.0, 0.0]),
 DenseVector([0.0, 0.0, 0.0, 0.0, 0.0]),
 DenseVector([0.0, 1.0, 1.0, 0.0, 0.0]),
 DenseVector([110.0, 1111.0, 1.0, 0.0, 0.0])]